In [48]:
# Define a custom type
struct MyData
    name::String
    values::Vector{Float64}
end

# Default text representation
Base.show(io::IO, data::MyData) = print(io, 
    "MyData with name: $(data.name) and $(length(data.values)) values")

# Specialized HTML representation
Base.show(io::IO, ::MIME"text/html", data::MyData) = print(io,
     "<strong>MyData</strong>: <br>Name: <em>$(data.name)</em><br>Values: $(data.values)")

# Specialized LaTeX representation
Base.show(io::IO, ::MIME"text/latex", data::MyData) = print(io, 
    "\\textbf{MyData}: \\textit{$(data.name)} \\\\ \\texttt{$(data.values)}")

In [49]:
mutable struct Document
    raw::String
end

mutable struct File{T <: Any}
    path::String
    File(path::String) = begin
        fending = split(path, ".")[2:end]
        new{Symbol(fending)}(path)
    end
end

In [50]:
# Define a struct for a unit of measurement
struct Unit{T}
    value::T
end

# Overload the multiplication operator
import Base.*

*(x::Number, u::Unit) = Unit(x * u.value)

# Define specific units
const Meter = Unit(1.0)  # 1 meter
const Second = Unit(1.0)  # 1 second

Unit{Float64}(1.0)

In [12]:
using StaticArrays
# our constructor:
mutable struct PointVector{N, T}
    values::SVector{N, T}
    function PointVector{N}(args::Any ...) where {N <: Any}
        n_values::Int64 = length(args)
        T = typeof(args[1])
        if n_values > 1 && n_values != N
            throw(DimensionMismatch())
        end
        buffer::SVector{N, T} = SA[args ...]
        new{N, T}(buffer)
    end
end

In [13]:
const Vector2{T} = PointVector{2}
const Vector3{T} = PointVector{3}

Vector3 (alias for PointVector{3})

In [20]:
println(Vector3(1, 5, 9))
println(Vector2(1, 2))

Vector3{Int64}([1, 5, 9])
Vector2{Int64}([1, 2])


In [39]:
mutable struct SoftDirectory{ACTIVE}
    files::Dict{String, String}
    path::String
    function SoftDirectory(path::String)
        new{false}(Dict{String, String}(), path::String)
    end
    function SoftDirectory(files::Dict{String, String}, path::String)
        new{true}(files, path)
    end
end

In [40]:
function mount(dir::SoftDirectory, path::String = dir.path)
    files = dir.files
    [begin 
        file_direc = path * "/$file"
        if isfile(file_direc)
            push!(files, file => read(file_direc, String))
        end
    end for file in readdir(path)]
    newdir = SoftDirectory(files, dir.path)
end

mount (generic function with 2 methods)

In [41]:
softdirec = SoftDirectory("/home/emmac/sampledir")

SoftDirectory{false}(Dict{String, String}(), "/home/emmac/sampledir")

In [42]:
mounted = mount(softdirec)

SoftDirectory{true}(Dict("test.txt" => ""), "/home/emmac/sampledir")

In [46]:
list_files(soft::SoftDirectory{true}) = println(keys(soft.files))
list_files(soft::SoftDirectory{false}) = throw("you have yet to initialize!")

list_files (generic function with 2 methods)

In [47]:
# list_files(softdirec)
list_files(mounted)

["test.txt"]
